In [1]:
import gurobipy as gp
from gurobipy import GRB

In [31]:
nodes = [1, 2, 3, 4, 5, 6, 7]

arcs, capacity = gp.multidict({
    (1, 2): 5 ,
    (1, 3): 6 ,
    (1, 4): 5 ,
    (2, 3): 2 ,
    (2, 5): 3 ,
    (3, 2): 2 ,
    (3, 4): 3 ,
    (3, 5): 3 ,
    (3, 6): 7 ,
    (4, 6): 5 ,
    (5, 6): 1 ,
    (5, 7): 8 ,
    (6, 5): 1 ,
    (6, 7): 7 ,
    (7,1): 1e21,
})

In [32]:
m = gp.Model('max_flow')

In [33]:
path = m.addVars(arcs, ub=capacity, vtype=GRB.CONTINUOUS, name='X')

In [40]:
Z = path[7,1]
m.ModelSense = GRB.MAXIMIZE
m.setObjective(Z)

In [41]:
flow = m.addConstrs(
    (path.sum(t, '*') - path.sum('*', t) == 0 for t in nodes),
    name='flow'
)

In [44]:
m.optimize()

print(f'Objective Value {m.ObjVal}')

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x3267aabc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 6 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4008000e+01   2.001600e+01   0.000000e+00      0s
       5    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.400000000e+01
Objective Value 14.0


In [42]:
m.update()

In [45]:
print(m.getConstrs())

[<gurobi.Constr flow[1]>, <gurobi.Constr flow[2]>, <gurobi.Constr flow[3]>, <gurobi.Constr flow[4]>, <gurobi.Constr flow[5]>, <gurobi.Constr flow[6]>, <gurobi.Constr flow[7]>]


In [47]:
for v in m.getVars():
    print(f'{v.VarName} <= {v.ub}')

X[1,2] <= 5.0
X[1,3] <= 6.0
X[1,4] <= 5.0
X[2,3] <= 2.0
X[2,5] <= 3.0
X[3,2] <= 2.0
X[3,4] <= 3.0
X[3,5] <= 3.0
X[3,6] <= 7.0
X[4,6] <= 5.0
X[5,6] <= 1.0
X[5,7] <= 8.0
X[6,5] <= 1.0
X[6,7] <= 7.0
X[7,1] <= inf


In [49]:
for v in m.getVars():
    if abs(v.X) > 0.00001:
        print(f'{v.VarName}: {v.X}')

X[1,2]: 5.0
X[1,3]: 4.0
X[1,4]: 5.0
X[2,3]: 2.0
X[2,5]: 3.0
X[3,5]: 3.0
X[3,6]: 3.0
X[4,6]: 5.0
X[5,7]: 7.0
X[6,5]: 1.0
X[6,7]: 7.0
X[7,1]: 14.0
